# Spectra of kinetic energy

In [ ]:
%pylab widget
#%pylab ipympl
#%pylab inline
from plotting_settings import *

import os
cwd = os.getcwd()
plot_dir = os.path.join(cwd, 'plots')
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

data_path = '/data/CoCo/'

In [ ]:
def get_avg_spectra(path, fn_format, dumps, nbins):
    k = np.zeros(nbins)
    power = np.zeros((2, nbins))
    
    for i, dmp in enumerate(dumps):
        spec_name = fn_format.format(dmp)
        data = np.loadtxt(os.path.join(path, spec_name), skiprows=5, max_rows=nbins)
        k += data[:, 0]
        power[0, :] += data[:, 1]
        data = np.loadtxt(os.path.join(path, spec_name), skiprows=5+nbins+4, max_rows=nbins)
        power[1, :] += data[:, 1]
    k /= len(dumps)
    power /= len(dumps)
    
    return k, power

k = {}
power = {}
run_lbls = {}
run_list = {}
dumps = range(100, 401)

codes = ('FLASH', 'MUSIC', 'PPMSTAR', 'PROMPI', 'SLH')

for res in (128, 256, 512):
    for code in codes:
        # We only have one full-length 512**3 run.
        if (res == 512) and (code not in ('PPMSTAR',)):
            continue
        
        rid = code+'-'+str(res)
        run_path = os.path.join(data_path, 'spectra', code, rid)
        fn_format = rid+'-'+'{:04d}.spec'
        run_lbls[rid] = r'{:s}, ${:d}^3$'.format(code, res)
        
        k[rid], power[rid] = get_avg_spectra(run_path, fn_format, dumps, res//2)
        print('{:s} done.'.format(rid))
        
    run_list[res] = [rid for rid in run_lbls.keys() if str(res) in rid]

In [ ]:
res = 256
runs = append(run_list[res], ['PPMSTAR-512'])
layer_lbl = ['cl', 'sl']

ifig = 1
for layer in (0, 1):
    close(ifig)
    figure(ifig, figsize=(width, 0.75*width), dpi=scrdpi)
    ifig += 1
    for i, rid in enumerate(runs):
        loglog(k[rid], power[rid][layer, :], ls=ls[i], color=lc[i], label=run_lbls[rid])
    
    kk = np.arange(1024)
    sr = kk**(-5./3.)
    if layer == 0:
        loglog(kk, 1.4e-5*sr/sr[6], ':', color='k', label=r'$k^{-5/3}$')
    xlim((1e0, 2e2))
    if layer == 0:
        ylim((1e-10, 1e-3))
    if layer == 1:
        ylim((1e-13, 1e-4))
    xlabel(r'$k$')
    ylabel(r'$\frac{1}{2} ||\, |\mathbf{\Psi}|\, ||^2$')
    if layer == 0:
        title('Convective layer')
    if layer == 1:
        title('Stable layer')
    legend(loc='lower left')
    tight_layout()
    savefig(os.path.join(plot_dir, 'spectra_{:s}_{:d}.pdf'.format(layer_lbl[layer], res)), dpi=savedpi)